In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser 

c:\Users\Win 11\Desktop\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## load text file
loader=TextLoader("langchain_sample.txt")
raw_docs=loader.load()
# Split text into document chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)
docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [5]:
## user query
query="How can i use langchain to build an application with memory and tools?"

In [6]:
### FAISS and Huggingface model Embeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(docs,embedding_model)
retriever=vectorstore.as_retriever(search_kwargs={"k":8})

In [7]:
## OpenAI Embedding
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)
retriever_openai=vectorstore_openai.as_retriever(search_kwargs={"k":8})

In [9]:
## prompt and use the llm
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001651C870710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001651B70B800>, root_client=<openai.OpenAI object at 0x000001651C872150>, root_async_client=<openai.AsyncOpenAI object at 0x000001651C870650>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [10]:
# Prompt Template
prompt = PromptTemplate.from_template("""
    You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question."
    User Question: "{question}",
    Documents:
    {documents}
    Instructions:
    - Think about the relevance of each document to the user's question.
    - Return a list of document indices in ranked order, starting from the most relevant.

    Output format: comma-separated document indices (e.g., 2,1,3,0,...)
    """)


In [11]:
retrieved_docs=retriever.invoke(query)
retrieved_docs

[Document(id='68671e22-621f-4163-989a-365b48f58cd0', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='65a788e0-3a06-4c83-938d-8a629987fcca', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='b5f41759-7a98-48fd-8e46-41b65e714235', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [12]:
chain=prompt| llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question."\n    User Question: "{question}",\n    Documents:\n    {documents}\n    Instructions:\n    - Think about the relevance of each document to the user\'s question.\n    - Return a list of document indices in ranked order, starting from the most relevant.\n\n    Output format: comma-separated document indices (e.g., 2,1,3,0,...)\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001651C870710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001651B70B800>, root_client=<openai.OpenAI object at 0x000001651C872150>, root_async_client=<openai.AsyncOpenAI object at 0x000001651C870650>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [16]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)
doc_lines

['1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and comp

In [17]:
formatted_docs

'1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed ind

In [18]:
response=chain.invoke({"question":query,"documents":formatted_docs})
response

'2,1,4,3,5,6'

In [19]:
# Step 5: Parse and rerank\n",
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[1, 0, 3, 2, 4, 5]

In [20]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]
reranked_docs

[Document(id='65a788e0-3a06-4c83-938d-8a629987fcca', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='68671e22-621f-4163-989a-365b48f58cd0', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='67e3bef4-1b5a-4f31-90c1-f28593f9ab4c', metadata={'source': 'langchain_sample.txt'}, page_content='FAISS is a popular library used for fast approximate nearest neighbor search 

In [21]:
# Step 6: Show results
print("📊 Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"Rank {i}:\n{doc.page_content}")

📊 Final Reranked Results:
Rank 1:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.
Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.
Rank 3:
FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with 